In [ ]:
# 足データを想定
# entry_price: numpy array。買い指値。i番目の足の最後で価格entry_price[i]で指値を出し、i + 1番目の足の最後でキャンセルする想定。
# lo: numpy array。低値
# 戻り値: y[i]は、i番目の足の最後から執行を始めた場合に、実際に約定する価格
# 計算が遅いので高速化のためにnumbaを使っている。numbaを除くと遅いけど同じ動作。

@numba.njit
def calc_force_entry_price(entry_price=None, lo=None, pips=None):
   y = entry_price.copy()
   y[:] = np.nan
   for i in range(entry_price.size):
       for j in range(i + 1, entry_price.size):
           if round(lo[j] / pips) < round(entry_price[j - 1] / pips):
               y[i] = entry_price[j - 1]
               break
   return y

In [ ]:
df['buy_fep'] = calc_force_entry_price(
    entry_price=df['buy_price'].values,
    lo=df['lo'].values,
    pips=pips,
)

# calc_force_entry_priceは入力をマイナスにすれば売りに使える
df['sell_fep'] = -calc_force_entry_price(
    entry_price=-df['sell_price'].values,
    lo=-df['hi'].values, # 売りのときは高値
    pips=pips,
)

In [ ]:
df['y_buy'] = df['sell_fep'].shift(-t) / df['buy_fep'] - 1 - 2 * fee
df['y_sell'] = -(df['buy_fep'].shift(-t) / df['sell_fep'] - 1) - 2 * fee